In [1]:
import requests
from bs4 import BeautifulSoup
import requests 
import json
import os
from dotenv import load_dotenv
import pandas as pd
from pandas import json_normalize
import tweepy
import time
import numpy as np
load_dotenv()

True

In [2]:
apikey = os.getenv("apikey") #apikey de lastfm

#### Importación el dataset que hemos creado en la limpieza 
- 500 albumes de la lista de Rolling Stone + los usuarios y reproducciones que han escuchado estos discos

In [3]:
stone_500_rich = pd.read_csv("../data/stone_500_rich.csv") 

In [4]:
stone_500_rich.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners
28,29,1968,The Beatles (“The White Album”),The Beatles,Album,4.4,rock,13125966,567654
323,324,2002,A Rush of Blood to the Head,Coldplay,Album,4.1,pop,59377635,2809272
488,489,1991,Back to Mono (1958–1969),Phil Spector,Album + Compilation,0.0,pop,5594,184


# Spotify Most streamed
- scrapping

[charset: most streamed albums](https://chartmasters.org/spotify-most-streamed-albums/?y=alltime)
Last updated on 11/06/21

In [5]:
url_spotify = "https://chartmasters.org/spotify-most-streamed-albums/?y=alltime"
html_spotify = requests.get(url_spotify)
soup_spotify = BeautifulSoup(html_spotify.content,"html.parser")

In [6]:
tablas_spotify = soup_spotify.findAll("table")

In [7]:
spotify = tablas_spotify[1]

In [8]:
spotify.find_all("tr")[3] #cuerpo

<tr><td>3</td><td class="Image"><img alt="Dua" height="50" lipa="" src="https://i.scdn.co/image/ab67616d0000b2731764e1a1b94e887206782640" width="50"/></td><td>Dua Lipa</td><td>Dua Lipa (Complete Edition)</td><td class="numbers">8,771,399,749</td><td class="numbers">3,456,568</td><td class="numbers">8,757,275</td></tr>

In [9]:
spotify.find_all("th") #cabeceros

[<th>Rank</th>,
 <th>Cover</th>,
 <th>Artist</th>,
 <th>Album</th>,
 <th>Total Streams</th>,
 <th>Daily Streams</th>,
 <th>EAS</th>]

In [10]:
#función que extrae la información de una tabla.
def data_tab(tab):
    Top_alb = []
    for s in tab.find_all("tr"):
        fila = [elemento for elemento in s.find_all("td")]
        if len(fila)>1:
            spot_dicc = {"Rank" : int(fila[0].text),
                        "Artist": fila[2].text.strip(),
                         "Album": fila[3].text.strip(),
                         "Total" :int(fila[4].text.replace(",","")),
                         "EAS" : int(fila[6].text.replace(",","")) #Equivalent Album Sales
                        }
            Top_alb.append(spot_dicc)
    return Top_alb

In [11]:
tab = spotify
Top_alb_spotify = data_tab(tab)
Top_alb_spotify[1]

{'Rank': 2,
 'Artist': 'Post Malone',
 'Album': 'beerbongs & bentleys',
 'Total': 9418695221,
 'EAS': 9403528}

### Spotify most streamed DataFrame

In [12]:
Spotify_500 = pd.DataFrame(Top_alb_spotify)
Spotify_500.head(5)

,Rank,Artist,Album,Total,EAS
0,1,Ed Sheeran,÷ (Deluxe),11179284635,11161283
1,2,Post Malone,beerbongs & bentleys,9418695221,9403528
2,3,Dua Lipa,Dua Lipa (Complete Edition),8771399749,8757275
3,4,Post Malone,Hollywood's Bleeding,8671157847,8657195
4,5,Ed Sheeran,x (Wembley Edition),8424703606,8411137


In [13]:
Spotify_500.to_csv("../data/Spotify_500.csv",index = False) 

### Stone DataFrame enriched whith Spotify DataFrame

In [14]:
stone_last_spot= stone_500_rich.merge(Spotify_500, how='left', on = ["Album","Artist"])

In [15]:
stone_last_spot.Rank.fillna(0, inplace=True)
stone_last_spot.Total.fillna(0, inplace=True)
stone_last_spot.EAS.fillna(0, inplace=True)

In [16]:
stone_last_spot[["Rank","Total","EAS"]] = stone_last_spot[["Rank","Total","EAS"]].astype("int64")

In [17]:
stone_last_spot.to_csv("../data/Spotify_500.csv",index = False) 

# LastFM 50 Top Artists
- [api](https://www.last.fm/api/show/chart.getTopArtists)
* Conseguimos la lista de los 50 Artistas más escuchados en LastFM para comparar con los artistas de la lista de RS

In [18]:
url_last50 = f"http://ws.audioscrobbler.com/2.0/?method=chart.gettopartists&api_key={apikey}&format=json"
req_last50 = requests.get(url_last50).json()
last_50 = req_last50["artists"]["artist"]
last_top50 = pd.DataFrame.from_dict(last_50)

In [19]:
last_top50.sample(3)

,name,playcount,listeners,mbid,url,streamable,image
45,J. Cole,82401648,2116190,875203e1-8e58-4b86-8dcb-7190faf411c5,https://www.last.fm/music/J.+Cole,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
41,Beyoncé,212788510,4172757,859d0860-d480-4efd-970c-c05d5f1776b8,https://www.last.fm/music/Beyonc%C3%A9,0,[{'#text': 'https://lastfm.freetls.fastly.net/...
32,Justin Bieber,99039319,2340654,e0140a67-e4d1-4f13-8a01-364355bee46e,https://www.last.fm/music/Justin+Bieber,0,[{'#text': 'https://lastfm.freetls.fastly.net/...


In [20]:
last_top50.rename(columns={"name": "Artist"}, inplace = True)

In [21]:
last_top50[["playcount","listeners"]] = last_top50[["playcount","listeners"]].astype("int64")

In [22]:
last_top50.to_csv("../data/otras/last_top50_image.csv",index = False) #guardada para mí

In [23]:
top_50_last = last_top50[["Artist","playcount","listeners"]]

In [24]:
top_50_last.to_csv("../data/top_50_last.csv",index = False) 

In [25]:
top_50_last.sample(2)

,Artist,playcount,listeners
6,Coldplay,430303065,6098162
10,Adele,157187423,3461005


In [26]:
#stone_500_rich_art = stone_500_rich.groupby("Artist").agg({"Album":"count","Rating":"mean"}).reset_index()
#stone_rich_top_50 = stone_500_rich_art.merge(top_50_last,how="left",on = "Artist")
#stone_rich_top_50

In [27]:
stone = pd.read_csv("../data/stone.csv")

In [28]:
stone_red = stone[["Number","Artist","Album","Rating","Genre"]]

In [29]:
stone_Artist = stone_red.groupby(["Artist"]).agg({"Number":"min","Album":"count","Rating":"mean"}).sort_values("Number", ascending = True).reset_index()
stone_Artist.Rating = round(stone_Artist.Rating, 2)

In [30]:
stone_Artist_last = stone_Artist.merge(top_50_last, how = "left", on= "Artist")

In [31]:
stone_Artist_last.playcount.fillna(0,inplace = True)
stone_Artist_last.listeners.fillna(0,inplace = True)
stone_Artist_last[["playcount","listeners"]] = stone_Artist_last[["playcount","listeners"]].astype("int64")

In [32]:
stone_Artist_last.head(3)

,Artist,Number,Album,Rating,playcount,listeners
0,Marvin Gaye,1,3,4.32,0,0
1,The Beach Boys,2,3,4.15,0,0
2,Joni Mitchell,3,4,4.36,0,0


In [33]:
stone_Artist_last.to_csv("../data/stone_Artist_last.csv",index = False)

# Top Sells
#### The 50 best selling albums of all time in USA. 
[top50](https://www.businessinsider.com/50-best-selling-albums-all-time-2016-9)
    - Travis Clark Updated Sep 22, 2021, 8:18 PM

In [34]:
url_insider = "https://www.businessinsider.com/50-best-selling-albums-all-time-2016-9"
html_insider = requests.get(url_insider)
soup_insider = BeautifulSoup(html_insider.content,"html.parser")

In [35]:
tags_insider = soup_insider.find_all("div", {"class": "slide-layout"})

In [36]:
tags_insider[0].find("h2").getText()

'50. Led Zeppelin —\xa0"Led Zeppelin II"'

In [37]:
tags_insider[0].find_all("p")[0].getText()

'Certified units: 12\xa0million'

In [38]:
def data_scrap(tags):
    lista = []
    for t in tags:
        dicc_t = {"art_alb": t.find("h2").getText(),
                 "certified units" : t.find("p").getText()}
        lista.append(dicc_t)
    return lista

In [39]:
insider_data = data_scrap(tags_insider)
insider_data[0]

{'art_alb': '50. Led Zeppelin —\xa0"Led Zeppelin II"',
 'certified units': 'Certified units: 12\xa0million'}

In [40]:
insider = pd.DataFrame(insider_data)
insider.head(2)

,art_alb,certified units
0,"50. Led Zeppelin — ""Led Zeppelin II""",Certified units: 12 million
1,"49. Kenny Rogers — ""Kenny Rogers' Greatest Hits""",Certified units: 12 million


In [41]:
#limpieza nuevo dataFrame
insider["Rank"] = insider["art_alb"].str.extract(r"(^\d+)")
insider["Millions"] = insider["certified units"].str.extract(r"(\d+)")
insider["art_alb"] = insider["art_alb"].str.replace(" – "," — ",regex = True)
insider["Artist"] = insider["art_alb"].str.extract(r"(^\d+.\s(.+?)—)")[1]
insider["Album"] = insider["art_alb"].str.extract(r"(—\s(.+?)$)")[1]
insider["Album"] = insider["Album"].str.replace('"',"")
insider["Artist"] = insider["Artist"].str.strip()
insider["Album"] = insider["Album"].str.strip()

In [42]:
insider.head(3)

,art_alb,certified units,Rank,Millions,Artist,Album
0,"50. Led Zeppelin — ""Led Zeppelin II""",Certified units: 12 million,50,12,Led Zeppelin,Led Zeppelin II
1,"49. Kenny Rogers — ""Kenny Rogers' Greatest Hits""",Certified units: 12 million,49,12,Kenny Rogers,Kenny Rogers' Greatest Hits
2,"48. Kenny G — ""Breathless""",Certified units: 12 million,48,12,Kenny G,Breathless


In [43]:
insider.to_csv("../data/otras/top_sells_certified.csv",index=False) #guardado para mí

In [44]:
insider[["Millions","Rank"]] = insider[["Millions","Rank"]].astype("int64")
top_sells = insider[["Rank","Artist","Album","Millions"]]
top_sells= top_sells.sort_values("Millions",ascending = False)

In [45]:
top_sells.head(3)

,Rank,Artist,Album,Millions
49,1,Eagles,Their Greatest Hits (1971-1975),38
48,2,Michael Jackson,Thriller,34
47,3,Eagles,Hotel California,26


In [46]:
top_sells["Album"] = top_sells["Album"].str.replace(r"(^The Beatles..The White\s.*)","The Beatles",regex = True)

In [47]:
top_sells.sample(3)

,Rank,Artist,Album,Millions
16,34,Simon & Garfunkel,Simon & Garfunkel's Greatest Hits,14
9,41,The Chicks,Wide Open Spaces,13
13,37,Bruce Springsteen,Bruce Springsteen & E Street Band Live 1975-'85,13


In [48]:
top_sells.to_csv("../data/top_sells.csv",index=False)

In [49]:
top_sell_rs = top_sells.merge(stone_500_rich, how = "left", on=["Artist","Album"])
top_sell_rs.head(3)

,Rank,Artist,Album,Millions,Number,Year,Type,Rating,Gen,playcount,listeners
0,1,Eagles,Their Greatest Hits (1971-1975),38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Michael Jackson,Thriller,34,12.0,1982.0,Album,4.40,pop,14012537.0,1305900.0
2,3,Eagles,Hotel California,26,118.0,1976.0,Album,4.45,rock,3433264.0,316205.0


In [50]:
top_sell_rs.Number.fillna(0,inplace = True)
top_sell_rs.Year.fillna(0,inplace = True)
top_sell_rs.Type.fillna("unknown",inplace = True)
top_sell_rs.Rating.fillna(0,inplace = True)
top_sell_rs.Gen.fillna("none",inplace = True)
top_sell_rs.playcount.fillna(0,inplace = True)
top_sell_rs.listeners.fillna(0,inplace = True)
top_sell_rs[["Number","Year","playcount","listeners"]] = top_sell_rs[["Number","Year","playcount","listeners"]].astype("int64")

In [51]:
top_sell_rs.sample(3)

,Rank,Artist,Album,Millions,Number,Year,Type,Rating,Gen,playcount,listeners
8,9,Garth Brooks,Double Live,21,0,0,unknown,0.00,none,0,0
35,39,Pearl Jam,Ten,13,160,1991,Album,4.15,grunge,34102474,1808072
22,20,Bee Gees,Saturday Night Fever (Soundtrack),15,0,0,unknown,0.00,none,0,0


In [52]:
top_sell_rs.to_csv("../data/top_sell_rs.csv",index=False)

In [53]:
stone_last_spot_sells = stone_last_spot.merge(top_sells,how = "left", on=["Artist","Album"])

In [54]:
stone_last_spot_sells = stone_last_spot_sells.rename(columns = {"Rank_x":"Spotify","Rank_y":"Sells"})

In [55]:
stone_last_spot_sells.Sells.fillna(0,inplace = True)
stone_last_spot_sells.Millions.fillna(0,inplace = True)

In [56]:
stone_last_spot_sells.head(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners,Spotify,Total,EAS,Sells,Millions
0,1,1971,What's Going On,Marvin Gaye,Album,4.65,r&b,5258671,546458,0,0,0,0.0,0.0
1,2,1966,Pet Sounds,The Beach Boys,Album,4.45,rock,13411664,543630,0,0,0,0.0,0.0
2,3,1971,Blue,Joni Mitchell,Album,4.45,folk,9405283,537629,0,0,0,0.0,0.0


In [57]:
stone_last_spot_sells[["Sells","Millions"]] = stone_last_spot_sells[["Sells","Millions"]].astype("int64")

In [58]:
stone_last_spot_sells.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners,Spotify,Total,EAS,Sells,Millions
408,409,1970,Workingman's Dead,Grateful Dead,Album,4.5,rock,1574799,166164,0,0,0,0,0
469,470,1998,400 Degreez,Juvenile,Album,3.0,hip-hop,633794,110802,0,0,0,0,0
379,380,1959,Mingus Ah Um,Charles Mingus,Album,4.4,jazz,2690789,309755,0,0,0,0,0


In [59]:
stone_last_spot_sells["In_Spot"] = np.where(stone_last_spot_sells["Spotify"],"Yes","No")
stone_last_spot_sells["In_Sells"] = np.where(stone_last_spot_sells["Sells"],"Yes","No")

In [60]:
stone_last_spot_sells.to_csv("../data/stone_last_spot_sells.csv",index=False)

## Resumen por artista de la lista de Rolling Stone

In [61]:
stone_500_rich.sample(3)

,Number,Year,Album,Artist,Type,Rating,Gen,playcount,listeners
272,273,1979,Entertainment!,Gang of Four,Album,4.75,punk,5961866,432185
438,439,1970,Sex Machine,James Brown,Album,4.35,funk,394155,48731
183,184,1983,She's So Unusual,Cyndi Lauper,Album,4.50,pop,4871988,797374


In [62]:
RS_Artists = stone_500_rich.groupby("Artist").agg({"Album":"count","playcount":"sum","listeners":"sum"}).reset_index()

In [63]:
RS_Artists.sample(3)

,Artist,Album,playcount,listeners
148,Kraftwerk,1,420273,25692
120,Ike & Tina Turner,1,176732,52590
238,Robert Johnson,1,1742686,302751


In [64]:
RS_Artists.to_csv("../data/RS_Artists.csv",index = False) 